# **Installing all of the packages**

In [1]:
!pip install -U accelerate bitsandbytes peft transformers datasets trl git-lfs wandb flash-attn sql-metadata scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 96.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [2]:
import os
import torch
import pandas as pd
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, TaskType
from datasets import load_dataset
from sql_metadata import Parser
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from tqdm import tqdm

# **Connect to wandb**

In [3]:
os.environ["WANDB_PROJECT"]="deepseek_full_finetuning"

# **Loading the model**

In [4]:
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_has_fp16_weight = True,
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant = True
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    #attn_implementation="flash_attention_2",
    torch_dtype = torch.float16,
    device_map='auto',
)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaLinearScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear4bit(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

# **Loading the dataset**

In [8]:
data_files = {"train": "train/full_finetuning_dataset.csv", "validation": "validation/validation_dataset_formatted.csv"}
dataset = load_dataset('csv', data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

**Filtering rows with max tokens**

In [9]:
def formatting_prompts_func(training_dataset):
  output_texts = []
  for i in range(len(training_dataset['question'])):
    question = training_dataset['question'][i]
    query = training_dataset['query'][i]
    database_schema = training_dataset['database_schema'][i]
    user_message = f"""Given the following SQL tables, your job is to generate the Sqlite SQL query given the user's question.
Put your answer inside the ```sql and ``` tags.
{database_schema}
###
Question: {question}
"""
    assitant_message = f"""
```sql
{query} ;
```
<|EOT|>
"""
    messages = [
      {"role": "user", "content": user_message},
      {"role": "assistant", "content": assitant_message},
      ]
    text = tokenizer.apply_chat_template(messages, tokenize=False)
    output_texts.append(text)
  return output_texts

In [10]:
response_template = "### Response:" #deepseek
# response_template = "[/INST]" #mistral"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# **Training Config**

In [11]:
lora_r = 64
lora_alpha = 32
lora_dropout = 0.1
output_dir = "./SFT_SQL"
num_train_epochs = 3
bf16 = False
fp16 = True
overwrite_output_dir = True
per_device_train_batch_size = 1
per_device_eval_batch_size = 2
gradient_accumulation_steps = 8
gradient_checkpointing = True
evaluation_strategy = "steps"
learning_rate = 5e-5
weight_decay = 0.01
lr_scheduler_type = "cosine"
warmup_ratio = 0.01
max_grad_norm = 0.3
group_by_length = True
auto_find_batch_size = False
save_steps = 50
logging_steps = 50
load_best_model_at_end= False
packing = False
save_total_limit=3
neftune_noise_alpha=5
#report_to="wandb"
max_seq_length = 1100 #set based on the maximum number of queries

In [12]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
        "q_proj",
        "v_proj",
        "k_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head"
    ],
    task_type=TaskType.CAUSAL_LM,
)

In [13]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    num_train_epochs=num_train_epochs,
    load_best_model_at_end=load_best_model_at_end,
    per_device_train_batch_size=per_device_train_batch_size,
    evaluation_strategy=evaluation_strategy,
    max_grad_norm = max_grad_norm,
    auto_find_batch_size = auto_find_batch_size,
    save_total_limit = save_total_limit,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    bf16=bf16,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    #report_to=report_to,
    neftune_noise_alpha= neftune_noise_alpha
)

In [14]:
os.environ["WANDB_MODE"] = "disabled"
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    tokenizer=tokenizer,
    args=training_arguments,
    max_seq_length=max_seq_length,
    packing=packing
)

Map:   0%|          | 0/3995 [00:00<?, ? examples/s]

Map:   0%|          | 0/433 [00:00<?, ? examples/s]

In [15]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/opt/conda/envs/pytorch/lib/python3.10/site-packages/trl/trainer/utils.py:141: UserWarning: Could not find response key `### Response:` in the following instance: <｜begin▁of▁sentence｜><｜begin▁of▁sentence｜>You are an AI programming assistant, utilizing the Deepseek Coder model, developed by Deepseek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer
### Instruction:
Given the following SQL tables, your job is to generate the Sqlite SQL query given the user's question.
Put your answer inside the ```sql and ``` tags.
CREATE TABLE `Demog` (
  GEOID INTEGER,
  INHABITANTS_K REAL,
  INCOME_K REAL,
  A_VAR1 REAL,
  A_VAR2 REAL,
  A_VAR3 REAL,
  A_VAR4 REAL,
  A_VAR5 REAL,
  A_VAR6 REAL,
  A_VAR7 REAL,
  A_

Step,Training Loss,Validation Loss


/opt/conda/envs/pytorch/lib/python3.10/site-packages/trl/trainer/utils.py:141: UserWarning: Could not find response key `### Response:` in the following instance: <｜begin▁of▁sentence｜><｜begin▁of▁sentence｜>You are an AI programming assistant, utilizing the Deepseek Coder model, developed by Deepseek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer
### Instruction:
Given the following SQL tables, your job is to generate the Sqlite SQL query given the user's question.
Put your answer inside the ```sql and ``` tags.
CREATE TABLE `sqlite_sequence` (
  name ,
  seq 
);
Sample rows from `sqlite_sequence`:
Categories, 8
Customers, 91
Employees, 10

CREATE TABLE `Categories` (
  CategoryID INTEGER,
  CategoryName TEXT,
  Description TEXT
);
Sample rows from `Categories`:
1, Beverages, Soft drinks, coffees, teas, beers, and ales
2, Condiments, S

KeyboardInterrupt: 

In [ ]:
output_dir = os.path.join("./", "final_checkpoint")
trainer.model.save_pretrained(output_dir)